In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import imutils

In [2]:
img_yes = '../data/yes/'
img_no = '../data/no/'

yes_files = os.listdir(img_yes)
no_files = os.listdir(img_no)

yes_images, no_images = [], []
print('[!] Reading files with positive diagnostic...')
for file in yes_files:
    yes_images.append(plt.imread(f'{img_yes}{file}'))
print('[!] Positive files read!')

print('[!] Reading files with negative diagnostic...')
for file in no_files:
    no_images.append(plt.imread(f'{img_no}{file}'))
print('[!] Negative files read')

all_images = yes_images + no_images

[!] Reading files with positive diagnostic...
[!] Positive files read!
[!] Reading files with negative diagnostic...
[!] Negative files read


In [3]:
THRESH = 30
MAX_VALUE = 255

def imgs_to_grayscale(img_list):
    for i, img in enumerate(img_list):
        if len(img.shape) == 3:
            img_list[i] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def threshold_imgs(img_list):
    for i, img in enumerate(img_list):
        _, img_list[i] = cv2.threshold(img, THRESH, MAX_VALUE, cv2.THRESH_BINARY)

def gaussian_blur(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.GaussianBlur(img, (5, 5), 0) 

def get_contour(img_list_grey, img_list):
    for img_grey, img in zip(img_list_grey, img_list):
        cont, _ = cv2.findContours(image=img_grey, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
        img_copy = img.copy()
        cv2.drawContours(image=img_copy, contours=cont, contourIdx=-1, color=(0, 255, 0), thickness=4, lineType=cv2.LINE_AA)
        cv2.imshow("Image", img_copy)
        cv2.waitKey()  

def draw_img(img_list, index):
    if index < len(img_list):
        cv2.imshow("Brain image", img_list[index])
        cv2.waitKey()

In [4]:
all_images_grey = all_images.copy()
draw_img(all_images, 0)
imgs_to_grayscale(all_images_grey)
draw_img(all_images_grey, 0)  
threshold_imgs(all_images_grey)
draw_img(all_images_grey, 0)  
gaussian_blur(all_images_grey)
draw_img(all_images_grey, 0)  
get_contour(all_images_grey, all_images)
draw_img(all_images_grey, 0)  